# Uvod v verjetnost in statistiko
V tem zvezku se bomo igrali z nekaterimi koncepti, o katerih smo že govorili. Veliko konceptov iz verjetnosti in statistike je dobro predstavljeno v glavnih knjižnicah za obdelavo podatkov v Pythonu, kot sta `numpy` in `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Naključne spremenljivke in porazdelitve
Začnimo z vlečenjem vzorca 30 vrednosti iz enakomerne porazdelitve od 0 do 9. Izračunali bomo tudi povprečje in varianco.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Za vizualno oceno, koliko različnih vrednosti je v vzorcu, lahko narišemo **histogram**:


In [ ]:
plt.hist(sample)
plt.show()

## Analiza dejanskih podatkov

Povprečje in varianca sta zelo pomembna pri analizi podatkov iz resničnega sveta. Naložimo podatke o igralcih baseballa z [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Tukaj za analizo podatkov uporabljamo paket [**Pandas**](https://pandas.pydata.org/). Kasneje v tem tečaju bomo več govorili o Pandas in delu s podatki v Pythonu.

Izračunajmo povprečne vrednosti za starost, višino in težo:


In [ ]:
df[['Age','Height','Weight']].mean()

Sedaj se osredotočimo na višino ter izračunajmo standardni odklon in varianco:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Poleg povprečja je smiselno pogledati tudi mediano in kvartile. Te lahko vizualiziramo z uporabo **škatlastega diagrama**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Lahko tudi naredimo škatlaste diagrame podskupin našega nabora podatkov, na primer razvrščene po vlogi igralca.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Opomba**: Ta diagram nakazuje, da so povprečne višine prvo baznih igralcev višje od višin drugobaznih igralcev. Kasneje se bomo naučili, kako lahko to hipotezo bolj formalno testiramo in kako dokazati, da so naši podatki statistično značilni za to.  

Starost, višina in teža so vse zvezne naključne spremenljivke. Kakšna mislite, da je njihova porazdelitev? Dobro je to ugotoviti z risanjem histograma vrednosti: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Normalna porazdelitev

Ustvarimo umetni vzorec teže, ki sledi normalni porazdelitvi z enakim povprečjem in varianco kot naši dejanski podatki:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Ker je večina vrednosti v resničnem življenju normalno porazdeljena, ne bi smeli uporabljati generatorja enakomerno porazdeljenih naključnih števil za generiranje vzorčnih podatkov. Tukaj je, kaj se zgodi, če poskušamo generirati teže z enakomerno porazdelitvijo (generirano z `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Intervali zaupanja

Izračunajmo intervale zaupanja za težo in višino bejzbol igralcev. Uporabili bomo kodo [iz te razprave na stackoverflow](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Testiranje hipotez

Raziščimo različne vloge v našem naboru podatkov bejzbolskih igralcev:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Preizkusimo hipotezo, da so prvi igralci na bazi višji od drugih igralcev na bazi. Najenostavnejši način za to je testiranje intervalov zaupanja:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Lahko vidimo, da se intervali ne prekrivajo.

Statistično bolj pravilno potrditev hipoteze izvedemo z uporabo **Studentove t-teste**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Dve vrednosti, ki jih vrne funkcija `ttest_ind`, sta:
* p-vrednost lahko razumemo kot verjetnost, da imata dve porazdelitvi enak povprečje. V našem primeru je zelo nizka, kar pomeni, da obstajajo trdni dokazi, da so prve baze višje.
* t-vrednost je vmesna vrednost normalizirane razlike povprečij, ki se uporablja v t-testu, in se primerja z mejno vrednostjo za dano stopnjo zaupanja.


## Simulacija normalne porazdelitve s centralnim limitnim izrekom

Generator psevdonaključnih števil v Pythonu je zasnovan tako, da nam daje enakomerno porazdelitev. Če želimo ustvariti generator za normalno porazdelitev, lahko uporabimo centralni limitni izrek. Da dobimo normalno porazdeljeno vrednost, bomo preprosto izračunali povprečje vzorca, ki je ustvarjen z enakomerno porazdelitvijo.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Korelacija in Zlobna Baseball Korporacija

Korelacija nam omogoča, da najdemo povezave med podatkovnimi zaporedji. V našem preprostemu primeru si zamislimo, da obstaja zlobna baseball korporacija, ki svojim igralcem plačuje glede na njihovo višino - višji kot je igralec, več denarja prejme. Predpostavimo, da je osnovna plača 1000 $, in dodatna nagrada od 0 do 100 $, odvisno od višine. Vzamemo resnične igralce iz MLB in izračunamo njihove namišljene plače:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Izračunajmo zdaj kovarianco in korelacijo teh zaporedij. `np.cov` nam bo dala t.i. **kovariančno matriko**, ki je razširitev kovariance na več spremenljivk. Element $M_{ij}$ kovariančne matrike $M$ je korelacija med vhodnima spremenljivkama $X_i$ in $X_j$, diagonalne vrednosti $M_{ii}$ pa so varianca $X_i$. Podobno nam bo `np.corrcoef` dala **korelacijsko matriko**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Korelacija enaka 1 pomeni, da obstaja močna **linearna povezava** med dvema spremenljivkama. Linearno povezavo lahko vizualno vidimo z risanjem vrednosti ene spremenljivke proti drugi:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Poglejmo, kaj se zgodi, če relacija ni linearna. Predpostavimo, da se je naše podjetje odločilo skriti očitno linearno odvisnost med višino in plačami ter uvedlo nekaj nelinearnosti v formulo, kot je `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

V tem primeru je korelacija nekoliko manjša, a je še vedno precej visoka. Zdaj, da bi razmerje naredili še manj očitno, bi morda želeli dodati nekaj dodatne naključnosti tako, da bi k plači dodali neko naključno spremenljivko. Poglejmo, kaj se zgodi:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Uganete, zakaj se pikice poravnajo v navpične črte tako?

Opazili smo korelacijo med umetno zasnovanim konceptom, kot je plača, in opazovano spremenljivko *višina*. Oglejmo si tudi, ali se dve opazovani spremenljivki, kot sta višina in teža, prav tako korelirata:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Na žalost nismo dobili nobenih rezultatov - samo nekaj nenavadnih vrednosti `nan`. To je posledica dejstva, da so nekatere vrednosti v naši seriji nedefinirane, predstavljene kot `nan`, kar tudi povzroči, da je rezultat operacije nedefiniran. Če pogledamo matriko, vidimo, da je problematičen stolpec `Weight`, ker je bila izračunana samokorelacija med vrednostmi `Height`.

> Ta primer prikazuje pomen **priprave podatkov** in **čiščenja**. Brez ustreznih podatkov ne moremo izračunati ničesar.

Uporabimo metodo `fillna` za zapolnitev manjkajočih vrednosti in izračunajmo korelacijo:


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Res je, da obstaja korelacija, vendar ne tako močna kot v našem umetnem primeru. Pravzaprav, če pogledamo razpršeni diagram ene vrednosti glede na drugo, bi bila povezava veliko manj očitna:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Zaključek

V tem zapisku smo se naučili, kako izvajati osnovne operacije na podatkih za izračun statističnih funkcij. Sedaj vemo, kako uporabiti ustrezno orodje matematike in statistike za dokazovanje nekaterih hipotez ter kako izračunati intervale zaupanja za poljubne spremenljivke, glede na vzorec podatkov.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Omejitev odgovornosti**:
Ta dokument je bil preveden z uporabo AI prevajalske storitve [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, upoštevajte, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvorno jeziku se šteje za avtoritativni vir. Za ključne informacije priporočamo strokovni človeški prevod. Ne odgovarjamo za morebitna nesporazuma ali napačne interpretacije, ki bi izhajali iz uporabe tega prevoda.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
